In [4]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
from fractions import Fraction
import warnings

warnings.filterwarnings('ignore')

In [5]:
# parameters
customer_id = '5'
formatted_attribute = 'Material'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')

buckets = """Curtains"""

regex_pattern=r'(?i)(\d+.\d*.Inch.+x.?\d+.?Inch)|(\d+.?\d+\/\d+.?Inch.?x.?\d+.?\d+\/\d+.?Inch)|(\d+.?\d+\/\d+\-Inch)|(\d+\.?\d*\s?.?Inch)|()'

value='%n/a%'
customer_name='%bedbathandbeyond%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [6]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)

In [7]:
df = query_from_file(file_name='query/bbby.sql', params=params)
df

attribute    value    customers_name  external_id  \
0       material  Acrylic  bedbathandbeyond     10135761   
1       material  Acrylic  bedbathandbeyond     12861176   
2       material  Acrylic  bedbathandbeyond     12861184   
3       material  Acrylic  bedbathandbeyond     12903731   
4       material  Acrylic  bedbathandbeyond     13638039   
...          ...      ...               ...          ...   
207424  material     Zinc  bedbathandbeyond  ="46112293"   
207425  material     Zinc  bedbathandbeyond     47739314   
207426  material     Zinc  bedbathandbeyond     60285706   
207427  material     Zinc  bedbathandbeyond     60285720   
207428  material     Zinc  bedbathandbeyond     64133294   

                                                                       long_desc  \
0                                                 Clear Acrylic Toilet Tank Tray   
1                                                           Zia Tumbler in Clear   
2                                                          Zia Toothbrush Holder   
3                                                            Zia Cotton Swab Jar   
4                                                        Droplet Clear Soap Dish   
...                                                                          ...   
207424                                                                      None   
207425                                  Moen® Danika Robe Hook in Brushed Nickel   
207426                           Rainier™ Toilet Paper Holder in Polished Chrome   
207427                                    Rainier™ Towel Ring in Polished Chrome   
207428  Moen® Tiffin 24-Inch Towel Bar with Press & Mark™in Mediterranean Bronze   

       resolution  
0        standard  
1        standard  
2        standard  
3        standard  
4        standard  
...           ...  
207424   standard  
207425   standard  
207426   standard  
207427   standard  
207428   standard  

[207429 rows x 6 columns]

In [10]:
# Find a list of values to iterate
df_strategy = query_from_file(file_name='./asset/easy.sql', params=params) #create query
df_strategy
list_values=df_strategy['value'].to_list()
list_values

['Acrylic',
 'Acrylic Blend',
 'Aluminum',
 'American Upland Cotton',
 'Aramid',
 'Bamboo',
 'Bamboo Blend',
 'Batiste',
 'Bone China',
 'Borosilicate Glass',
 'Brass',
 'Bronze',
 'Brushed Cotton',
 'Burlap',
 'Canvas',
 'Canvas Blend',
 'Carbon Fiber',
 'Carbon Steel',
 'Cast Iron',
 'Ceramic',
 'Chenille',
 'Chrome',
 'Clay',
 'Concrete',
 'Copper',
 'Cork',
 'Cotton',
 'Cotton Blend',
 'Crystal',
 'Denim',
 'Down',
 'Down Alternative',
 'Earthenware',
 'Egyptian Cotton',
 'Elastic',
 'Enamel',
 'Engineered Wood',
 'Fabric',
 'Faux Fur',
 'Faux Leather',
 'Faux Suede',
 'Faux Wool',
 'Fiberboard',
 'Fiberglass',
 'Flannel',
 'Fleece',
 'Foam',
 'Fur',
 'Gel Fiber',
 'Glass',
 'Gold',
 'Granite',
 'Hyacinth',
 'Iron',
 'Jersey',
 'Jute',
 'Laminate',
 'Latex',
 'Lead Crystal',
 'Leather',
 'Linen',
 'Linen Blend',
 'Lycra',
 'Marble',
 'Melamine',
 'Memory Foam',
 'Mesh',
 'Metal',
 'Microfiber',
 'Microfiber Blend',
 'Microfleece',
 'Microsuede',
 'Modal',
 'Mother of Pearl',
 'Musl

In [11]:
kwp=KeywordProcessor()
print(kwp.add_keywords_from_list(list_values))
# Extract Key Words
df['long_desc'].apply(lambda x: kwp.extract_keywords(x)) #

None


0               [Acrylic]
1                      []
2                      []
3                [Cotton]
4                      []
               ...       
207424                 []
207425           [Nickel]
207426    [Paper, Chrome]
207427           [Chrome]
207428           [Bronze]
Name: long_desc, Length: 207429, dtype: object

In [12]:
df['matches'] = df['long_desc'].apply(lambda x: kwp.extract_keywords(x))
df['matches'].explode().value_counts()

Brass                 5106
Stainless Steel       4877
Glass                 4826
Gold                  3948
Wool                  3556
                      ... 
Bone China               3
Lead Crystal             3
Faux Wool                3
Borosilicate Glass       1
Polyresin                1
Name: matches, Length: 106, dtype: int64

In [13]:
df

attribute    value    customers_name  external_id  \
0       material  Acrylic  bedbathandbeyond     10135761   
1       material  Acrylic  bedbathandbeyond     12861176   
2       material  Acrylic  bedbathandbeyond     12861184   
3       material  Acrylic  bedbathandbeyond     12903731   
4       material  Acrylic  bedbathandbeyond     13638039   
...          ...      ...               ...          ...   
207424  material     Zinc  bedbathandbeyond  ="46112293"   
207425  material     Zinc  bedbathandbeyond     47739314   
207426  material     Zinc  bedbathandbeyond     60285706   
207427  material     Zinc  bedbathandbeyond     60285720   
207428  material     Zinc  bedbathandbeyond     64133294   

                                                                       long_desc  \
0                                                 Clear Acrylic Toilet Tank Tray   
1                                                           Zia Tumbler in Clear   
2                                                          Zia Toothbrush Holder   
3                                                            Zia Cotton Swab Jar   
4                                                        Droplet Clear Soap Dish   
...                                                                          ...   
207424                                                                      None   
207425                                  Moen® Danika Robe Hook in Brushed Nickel   
207426                           Rainier™ Toilet Paper Holder in Polished Chrome   
207427                                    Rainier™ Towel Ring in Polished Chrome   
207428  Moen® Tiffin 24-Inch Towel Bar with Press & Mark™in Mediterranean Bronze   

       resolution          matches  
0        standard        [Acrylic]  
1        standard               []  
2        standard               []  
3        standard         [Cotton]  
4        standard               []  
...           ...              ...  
207424   standard               []  
207425   standard         [Nickel]  
207426   standard  [Paper, Chrome]  
207427   standard         [Chrome]  
207428   standard         [Bronze]  

[207429 rows x 7 columns]

In [18]:
df[df['matches'].astype(str)!='[]'][300:350]

attribute     value    customers_name external_id  \
839  material  Aluminum  bedbathandbeyond    45565052   
840  material  Aluminum  bedbathandbeyond    45565069   
842  material  Aluminum  bedbathandbeyond    45565083   
843  material  Aluminum  bedbathandbeyond    45565090   
846  material  Aluminum  bedbathandbeyond    45587900   
849  material  Aluminum  bedbathandbeyond    45682780   
852  material  Aluminum  bedbathandbeyond    45951985   
853  material  Aluminum  bedbathandbeyond    45960321   
854  material  Aluminum  bedbathandbeyond    45960338   
856  material  Aluminum  bedbathandbeyond    46070524   
858  material  Aluminum  bedbathandbeyond    46302663   
859  material  Aluminum  bedbathandbeyond    46302687   
860  material  Aluminum  bedbathandbeyond    46342812   
862  material  Aluminum  bedbathandbeyond    46568946   
863  material  Aluminum  bedbathandbeyond    46568953   
864  material  Aluminum  bedbathandbeyond    46568960   
865  material  Aluminum  bedbathandbeyond    46568977   
866  material  Aluminum  bedbathandbeyond    46568984   
867  material  Aluminum  bedbathandbeyond    46568991   
868  material  Aluminum  bedbathandbeyond    46569004   
869  material  Aluminum  bedbathandbeyond    46569011   
870  material  Aluminum  bedbathandbeyond    46569028   
871  material  Aluminum  bedbathandbeyond    46569035   
872  material  Aluminum  bedbathandbeyond    46569042   
875  material  Aluminum  bedbathandbeyond    46724083   
876  material  Aluminum  bedbathandbeyond    46799852   
878  material  Aluminum  bedbathandbeyond    46919021   
881  material  Aluminum  bedbathandbeyond    47085848   
882  material  Aluminum  bedbathandbeyond    47098725   
883  material  Aluminum  bedbathandbeyond    47101906   
884  material  Aluminum  bedbathandbeyond    47236752   
886  material  Aluminum  bedbathandbeyond    47237155   
889  material  Aluminum  bedbathandbeyond    47279575   
890  material  Aluminum  bedbathandbeyond    47279582   
891  material  Aluminum  bedbathandbeyond    47279599   
892  material  Aluminum  bedbathandbeyond    47279605   
893  material  Aluminum  bedbathandbeyond    47279629   
894  material  Aluminum  bedbathandbeyond    47279636   
895  material  Aluminum  bedbathandbeyond    47279643   
896  material  Aluminum  bedbathandbeyond    47279650   
897  material  Aluminum  bedbathandbeyond    47279667   
898  material  Aluminum  bedbathandbeyond    47279674   
899  material  Aluminum  bedbathandbeyond    47279681   
900  material  Aluminum  bedbathandbeyond    47287143   
901  material  Aluminum  bedbathandbeyond    47287150   
902  material  Aluminum  bedbathandbeyond    47287167   
903  material  Aluminum  bedbathandbeyond    47287174   
904  material  Aluminum  bedbathandbeyond    47287181   
924  material  Aluminum  bedbathandbeyond    47287679   
950  material  Aluminum  bedbathandbeyond    47456150   

                                                                                      long_desc  \
839                                         Wilton® Performance™ Aluminum 7-Inch Angel Food Pan   
840                                    Wilton® Performance™ Aluminum 9-Inch Deep Springform Pan   
842                                     Wilton® Performance™ Aluminum 6-Cavity Jumbo Muffin Pan   
843                                        Wilton® Performance™ Aluminum 6-Inch Deep Square Pan   
846                                            Paderno® Aluminum 13-Inch x 21-Inch Baking Sheet   
849         TITAN® NeverRust® Aluminum Double Roller Shower Curtain Hooks in Chrome (Set of 12)   
852                                         InterDesign® Rust Proof Aluminum Suction Sink Caddy   
853                                                  InterDesign® Large Aluminum Sink Protector   
854                                                  InterDesign® Small Aluminum Sink Protector   
856               Farberware® High Performance Nonstick Aluminum 17-Piece Cookware Set in Black   
858    

In [66]:
def na_to_gsheet(df):
    url = 'https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0'
    wks = 'n/a'
    non_matches = df[df['matches'].astype(str) == '[]']
    non_matches[curation_col] = r'n/a'
    gc = gcloud.Gsheets()
    gc.save_to_gsheet(url, wks, non_matches)

In [67]:
na_to_gsheet(df)

succesfully saved to worksheet:`n/a` 
 url = `https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0`


In [70]:
error

NameError: name 'error' is not defined

In [ ]:
matches=df
matches[curation_col] = matches['Max'].astype(str)+' in'
matches['external_id']=matches['external_id'].apply('="{}"'.format)
matches

In [ ]:
matches[0:50]

In [ ]:
## KWP
# matches[curation_col]=matches['matches']
# matches['external_id']=matches['external_id'].apply('="{}"'.format)
# matches

In [ ]:
error----Switch Max and Min when doing length and width

In [ ]:
gc = gcloud.Gsheets()

In [ ]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',matches)

# send to the folder for upload

In [ ]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload/Shawn'
    non_matches = df[df['matches'].astype(str) == '[]']
    non_matches[curation_col] = r'n/a'
    non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute} - match upload {buckets}.csv',index=False)

In [ ]:
looks_good('Bed Bath & Beyond', attribute, df, matches)